In [1]:
import pygame
import math

pygame.init()

width, height = 1000, 700
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

white = (255, 255, 255)
black = (0, 0, 0)
blue = (0, 0, 255)
red = (255, 0, 0)
purple = (128, 0, 128)

def calculate_color(intensity):
    
    normalized_intensity = max(min(intensity, 1), -1)

    if normalized_intensity < 0:
        blue_to_purple = (1 + normalized_intensity)
        red = int(blue_to_purple * 128)
        green = 0
        blue = int(255 - blue_to_purple * 127)
    else:
        purple_to_red = normalized_intensity
        red = int(128 + purple_to_red * 127)
        green = 0
        blue = int((1 - purple_to_red) * 128)

    return (red, green, blue)

class Slider:
    def __init__(self, x, y, w, h, min_val, max_val, label, is_frequency=False):
        self.rect = pygame.Rect(x, y, w, h)
        self.min_val = min_val
        self.max_val = max_val
        self.val = (min_val + max_val) / 2
        self.grabbing = False
        self.label = label
        self.is_frequency = is_frequency  # Indica se o slider é de frequência

    def draw(self, screen):
        
        pygame.draw.rect(screen, white, self.rect)
        handle_x = self.rect.x + self.get_handle_position() * self.rect.width

        pygame.draw.rect(screen, black, (handle_x - 5, self.rect.y, 10, self.rect.height))

        font = pygame.font.SysFont(None, 24)
        label_surf = font.render(f'{self.label}: {self.get_value():.2f}', True, white)
        screen.blit(label_surf, (self.rect.x, self.rect.y - 25))

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.grabbing = True
        elif event.type == pygame.MOUSEBUTTONUP:
            self.grabbing = False
        elif event.type == pygame.MOUSEMOTION:
            if self.grabbing:
                handle_x = max(self.rect.x, min(event.pos[0], self.rect.x + self.rect.width))
                self.set_handle_position((handle_x - self.rect.x) / self.rect.width)

    def get_handle_position(self):
        return (self.val - self.min_val) / (self.max_val - self.min_val)

    def get_value(self):
        handle_pos = self.get_handle_position()
        if self.is_frequency:
            if handle_pos <= 0.5:
                return handle_pos * 2 * 100  # Primeiros 50% (0-100)
            else:
                return 100 + (handle_pos - 0.5) * 2 * 900  # Últimos 50% (100-1000)
        else:
            return self.min_val + handle_pos * (self.max_val - self.min_val)

    def set_handle_position(self, pos):
        if self.is_frequency:
            if pos <= 0.5:
                self.val = pos * 2 * 100
            else:
                self.val = 100 + (pos - 0.5) * 2 * 900
        else:
            self.val = self.min_val + pos * (self.max_val - self.min_val)


amplitude_slider1 = Slider(56, 595, 200, 20, 0, 2, 'X Max Amplitude')
amplitude_slider2 = Slider(56, 645, 200, 20, 0, 2, 'Y Max Amplitude')

frequency_slider1 = Slider(281, 595, 200, 20, 1, 200, 'X Frequency')
frequency_slider2 = Slider(281, 645, 200, 20, 1, 200, 'Y Frequency')

lambda_slider_x = Slider(506, 595, 200, 20, 0.01, 500, 'Lambda X')
lambda_slider_y = Slider(506, 645, 200, 20, 0.01, 500, 'Lambda Y')

phase_diff_slider1 = Slider(761, 595, 200, 20, 0, 2 * math.pi, 'X Phase (rad)')
phase_diff_slider2 = Slider(761, 645, 200, 20, 0, 2 * math.pi, 'Y Phase (rad)')

rect_width, rect_height = 990, 150
rectangle = pygame.Rect(5, 545, rect_width, rect_height)


running = True
t = 0
pixel_size = 3

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        amplitude_slider1.handle_event(event)
        frequency_slider1.handle_event(event)
        phase_diff_slider1.handle_event(event)
        lambda_slider_x.handle_event(event)
        
        amplitude_slider2.handle_event(event)
        frequency_slider2.handle_event(event)
        phase_diff_slider2.handle_event(event)
        lambda_slider_y.handle_event(event)

    screen.fill(black)

    amp_1 = amplitude_slider1.get_value()
    freq_1 = frequency_slider1.get_value()
    lamb_1 = lambda_slider_x.get_value()
    phasediff_1 = phase_diff_slider1.get_value()
    
    amp_2 = amplitude_slider2.get_value()
    freq_2 = frequency_slider2.get_value()
    lamb_2 = lambda_slider_y.get_value()
    phasediff_2 = phase_diff_slider2.get_value()

    for x in range(0, width, pixel_size):
        for y in range(0, height, pixel_size):
            # Updated wave equations with phase and speed
            k1 = 2 * math.pi / lamb_1
            w1 = 2 * math.pi * freq_1
            
            k2 = 2 * math.pi / lamb_2
            w2 = 2 * math.pi * freq_2
            
            dx = amp_1 * math.sin(k1*x - w1*t + phasediff_1)
            dy = amp_2 * math.sin(k2*y - w2*t + phasediff_2)
            
            intensity = dx + dy
            color = calculate_color(intensity)
            pygame.draw.rect(screen, color, (x, y, pixel_size, pixel_size))


    pygame.draw.rect(screen, black, rectangle)
    
    amplitude_slider1.draw(screen)
    frequency_slider1.draw(screen)
    phase_diff_slider1.draw(screen)
    lambda_slider_x.draw(screen)
    
    amplitude_slider2.draw(screen)
    frequency_slider2.draw(screen)
    phase_diff_slider2.draw(screen)
    lambda_slider_y.draw(screen)


    pygame.display.flip()
    t += 0.001
    clock.tick(60)

pygame.quit()


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
